# Finite Thoughts Automaton

This notebook focuses on creating FTA a lower level automaton than STA which could be called a "machine model" for LM.

In [1]:
import os, sys, json
sys.path.append(os.path.realpath(f"{os.getcwd()}/../"))
#from autocog.lm import TfLM, OpenAI, Llama
#llama_path = lambda x: "/workspace/models/{}/ggml-model-{}.bin".format(*x)
#llm = Llama(**Llama.create(model_path=llama_path(('7B','q4_0')), n_ctx=2048), max_tokens=20)
llm = None

In [2]:
from autocog.fta.automaton import FiniteThoughtAutomaton as FTA
from autocog.fta.actions import Choose, Complete, Text

nonzero_digits = ['1','2','3','4','5','6','7','8','9']

fta = FTA()
subject   = fta.create(uid='subject',   cls=Text,     successors=['qualifier'],           text="the speed of light in "                        )
qualifier = fta.create(uid='qualifier', cls=Choose,   successors=['verb','verb','verb'],  choices=[ "km/s", "m/s", "km/h" ]                    )
verb      = fta.create(uid='verb',      cls=Text,     successors=['digits_1'],            text=" is "                                          )
digits_1  = fta.create(uid='digits_1',  cls=Complete, successors=['sep_1'],               length=3, seeds=nonzero_digits, stop=[',','.']       )
sep_1     = fta.create(uid='sep_1',     cls=Choose,   successors=['digits_2','digits_4'], choices=[ ',', '.' ]                                 )
digits_2  = fta.create(uid='digits_2',  cls=Complete, successors=['sep_2'],               length=3, seeds=['0']+nonzero_digits, stop=[',','.'] )
sep_2     = fta.create(uid='sep_2',     cls=Choose,   successors=['digits_3','digits_4'], choices=[ ',', '.' ]                                 )
digits_3  = fta.create(uid='digits_3',  cls=Complete, successors=['sep_3'],               length=3, seeds=['0']+nonzero_digits, stop=['.']     )
sep_3     = fta.create(uid='sep_3',     cls=Text,     successors=['digits_4'],            text="."                                             )
digits_4  = fta.create(uid='digits_4',  cls=Complete, successors=['eol'],                 length=2, seeds=['0']+nonzero_digits, stop=['\n']    )
eol       = fta.create(uid='eol',       cls=Text,     successors=[],                      text="\n"                                            )

from autocog.utility.pynb import wrap_graphviz
wrap_graphviz(fta.toGraphViz())